In [ ]:
import json
import numpy as np
import random
from tqdm.auto import tqdm
import itertools
import os
from copy import deepcopy
import matplotlib.pyplot as plt
from collections import defaultdict
import string
import shutil


In [2]:
def complete(s):
    if not s.endswith(">"):
        s = s + ">"
    if not s.startswith("<"):
        s = "<" + s
    return s

In [3]:
DATA_ROOT = "/data/locus/project_data/project_data2/chenwu2/creativity_data"

In [4]:
NUM_ENTITIES = 10
NUM_NODES = 9
HASH_STR_LEN = 10

In [5]:
pad_token = "<|endoftext|>"

In [ ]:
# downsampling train_inferred
for training_size in [10000]:
    print(f"training size: {training_size}")
    base_dataset_name = "line.{}.{}.{}.{}".format(NUM_ENTITIES, NUM_NODES, HASH_STR_LEN, training_size)
    dataset_name = "line_hybrid.{}.{}.{}.{}".format(NUM_ENTITIES, NUM_NODES, HASH_STR_LEN, training_size)
    # Copy the base dataset to the new dataset
    shutil.copytree(os.path.join(DATA_ROOT, base_dataset_name), os.path.join(DATA_ROOT, dataset_name))

    train_sequences = json.load(open(os.path.join(DATA_ROOT, base_dataset_name, "train.json"), "r", encoding='utf-8'))
    # Add the hybrid data to the train_sequences
    hybrid_sequences = []
    for train_sequence in train_sequences:
        input_text = train_sequence["input_text"]
        target_text = train_sequence["target_text"]
        # Repeat 1 times
        for _ in range(1):
            ntp_sequence = {
                "input_text": input_text,
                "target_text": target_text,
                "target_labels": target_text,
            }
            hybrid_sequences.append(ntp_sequence)
        # Repeat 3 times
        for _ in range(3):
            mtp_sequence = {
                "input_text": input_text,
                "target_text": input_text + pad_token * NUM_NODES * 3 + "</a>",
                "target_labels": target_text
            }
            hybrid_sequences.append(mtp_sequence)

    with open(os.path.join(DATA_ROOT, dataset_name, "train.json"), "w", encoding='utf-8') as f:
        json.dump(hybrid_sequences, f)
    